# Montar el Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
os.chdir('/content/gdrive/My Drive/TFM-MaskDetection/jetsonNano/') # Cambia al directorio donde se encuentra training.ipynb

# Instalación de PyTorch

In [ ]:
# Install prerequisites
!pip install boto3
!pip install pandas
!pip install urllib3

In [ ]:
# Clone repository
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "pytorch-ssd" in pathlib.Path.cwd().parts:
  while "pytorch-ssd" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('pytorch-ssd').exists():
  !git clone --depth 1 https://github.com/dusty-nv/pytorch-ssd.git

os.chdir('pytorch-ssd')

# Preparación del dataset y el modelo

In [ ]:
# Convert png into jpg
from PIL import Image
os.chdir('data/JPEGImages')
img_list = os.listdir()
for img_name in img_list:
  img = Image.open(img_name)
  img = img.convert('RGB')
  img.save(img_name[:-3]+'jpg')

os.chdir('../..')

In [ ]:
# Generate training and validation sub-datasets
os.chdir('data')
!python3 ../vision/datasets/generate_vocdata.py ./labels.txt
os.chdir('..')

In [ ]:
# Download SSD MobileNet v1
!wget https://nvidia.box.com/shared/static/djf5w54rjvpqocsiztzaandq1m3avr7c.pth -O models/mobilenet-v1-ssd-mp-0_675.pth
!pip3 install -v -r requirements.txt

# Entrenamiento

In [ ]:
# Train model
!python3 train_ssd.py --dataset-type=voc \
  --data=./data \
  --model-dir=./models \
  --epochs=100 \
  --pretrained-ssd=./models/mobilenet-v1-ssd-mp-0_675.pth                       # Empezar a entrenar un modelo nuevo
  #--resume=./models/<chechpoint>.pth                                           # Continuar entrenando un modelo

# Conversión al formato ONNX

In [ ]:
!python3 onnx_export.py --model-dir=./models